# Membership Inference Label Only Decision-Boundary Attack Examples

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/privML/privacy-evaluator/blob/main/notebooks/membership_inference_label_only_decision_boundary_attack.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/team1sprint5/notebooks/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/privML/privacy-evaluator/blob/main/notebooks/membership_inference_abel_only_decision_boundary_attack.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/main/notebooks/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


## Overview

In this notebook, we want to show you how to use the `privacy-evaluator` tool to perform the Membership Inference Attacks Black Box Label Only Decision-Boundary Attack on both a provided PyTorch and a provided Tensorflow model.

## Setup

First, you should set the notebook's runtime to use a GPU (e.g. if Colab is used go to ***Runtime > Change runtime type > Hardware accelerator***). Now we can install the `privacy-evaluator` package and import all needed modules.

In [ ]:
!pip3 install git+https://github.com/privML/privacy-evaluator

In [ ]:
import tensorflow as tf
import torch
import numpy as np

import tensorflow.python.ops.numpy_ops.np_config as np_config
np_config.enable_numpy_behavior()

import privacy_evaluator.models.torch.dcti.dcti as torch_dcti
import privacy_evaluator.models.tf.dcti.dcti as tf_dcti 

from privacy_evaluator.datasets.tf.cifar10 import TFCIFAR10
from privacy_evaluator.datasets.torch.cifar10 import TorchCIFAR10

from privacy_evaluator.classifiers.classifier import Classifier

from privacy_evaluator.attacks.membership_inference.label_only_decision_boundary import MembershipInferenceLabelOnlyDecisionBoundaryAttack
from privacy_evaluator.attacks.membership_inference import MembershipInferenceAttackAnalysis

from privacy_evaluator.attacks.membership_inference import MembershipInferenceAttackAnalysis

from privacy_evaluator.attacks.membership_inference.data_structures.attack_input_data import AttackInputData
from privacy_evaluator.attacks.membership_inference.data_structures.slicing import Slicing

## Conduct Membership Inference Attacks

Now we can start with conducting the Membership Inference Label Only Decision-Boundary Attacks. Therefore, we prepared two instances of the attack: one attacking a PyTorch model and attacking a TensorFlow model. For both attacks, we implemented a simple neural network trained on the CIFAR-10 dataset. For details about the provided network have a look at the following paper: https://www.scitepress.org/Papers/2018/67520/67520.pdf).

### PyTorch

We start the evaluation of the PyTorch version of the model.

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `torch.nn.CrossEntropyLoss`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize PyTorch model as a Classifier
target_model = Classifier(
    torch_dcti.load_dcti(), # PyTorch DCTI 
    loss=torch.nn.CrossEntropyLoss(reduction="none"), # Loss function of the PyTorch model
    nb_classes=TorchCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TorchCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

#### Load CIFAR10 Dataset

Before we can start to conduct the membership inference attacks, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the PyTorch model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TorchCIFAR10.numpy()

#### Perform Membership Inference Label Only Decision Boundary Attack

Last but not least, we want to perform a Label Only Decision Boundary Attack. We again need to train the attack model first. This attack is really time expensive, thus we only fit the attack on a very small set which is not realistic and should not be repeated in production.

In [ ]:
attack = MembershipInferenceLabelOnlyDecisionBoundaryAttack(target_model)
attack.fit(x_train[:100], y_train[:100], x_test[:20], y_test[:20], max_iter=5, max_eval=5, init_eval=5)
attack.attack(x_train[:20], y_train[:20])

#### Get human-readable attack statistics

For this attack we want a human-readable output, thus we convert it to a sting.

In [ ]:
output = attack.attack_output(
    x_train[:20], 
    y_train[:20], 
    x_train, 
    y_train, 
    x_test, 
    y_test,
    np.ones((20,))
)

str(output)

#### Explanation of the outcome:

Attack Model Accuracy:
The attack model accuracy specifies how well the membership attack model performs in predicting if a given data point was used for training the target model. Since we have a two-class classification problem that the attack model solves (member or non-member), the lowest possible accuracy is 50% (random guessing for each sample). The best accuracy is at 100% if the model predicts every data point is sees right as member or non-member.

Train-Test-Gap (difference):
If your model has a train-test-gap larger than 5%, this could be a sign that your model overfits. Overfitting can be beneficial for successful membership inference attacks [1]. Therefore, you might want to reduce it by introducing regularization methods in your training, or using specific privacy methods [2,3], such as Differential Privacy [4].

(For References, please see last box)

#### Perpare attack analysis

Next, we prepare our attack analysis. To initialize our attack analysis we define the Membership Inference Attack method we want to perform (in this case we use the `MembershipInferenceLabelOnlyDecisionBoundaryAttack`) and the Attack Input Data. The Attack Input Data consists of two different sets. The first contains the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The second contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model.

In [ ]:
attack_analysis = MembershipInferenceAttackAnalysis(
    MembershipInferenceLabelOnlyDecisionBoundaryAttack, 
    AttackInputData(
        x_train[:100], 
        y_train[:100], 
        x_test[:20], 
        y_test[:20]
    )
)

#### Define the slicing

Now we can define the slicing for our analysis. The slicing defines how the data will be sliced. Each slice will then be analysed by the attack separately, so see if the inference works better on certain parts of data than on others. 

In [ ]:
slicing = Slicing(
    entire_dataset=True, 
    by_class=True, 
    by_classification_correctness=True
)

#### Perform Membership Inference Attack Analysis

Finally, we can perform our Membership Inference Attack Analysis. Therefore, we input the target model, the data that should be analysed, the membership labels (i.e. the labels which correctly describe if a data point is a member of the training dataset or not) and the splicing specification into the `analyse()` method. As a result, we get for each slice the indices of the corresponding data points, a human-readable description of the slice and the advantage score of the Membership Inference Attack (for more details about the advantage score, please read the following paper: https://arxiv.org/abs/1709.01604).

In [ ]:
result = attack_analysis.analyse(
    target_model, 
    np.concatenate((x_train[:100], x_test[:100])), 
    np.concatenate((y_train[:100], y_test[:100])), 
    np.concatenate((np.ones(len(x_train[:100])), np.zeros(len(x_test[:100])))), 
    slicing
)

print("\n".join((str(r) for r in result)))

#### Explanation of the outcome:
##### Attacker Advantage:
The attacker advantageis a score that relies on comparing the model output on member and non-member data points. The model outputs are probability values over all classes, and they are often different on member and non-member data points. Usually, the model is more confident on member data points, because it has seen them during training. When trying to find a threshold value to tell apart member and non-member samples by their different model outputs, the attacker has interest in finding the best ratio between false positives “fpr” (non-members that are classified as members) and true positives “tpr” (members that are correctly identifies as members). 

This best ratio is calculated as the max(tpr-fpr) over all threshold values and represents the attacker advantage. 

##### Slicing: Incorrectly classified:
It is normal that the attacker is more successful to deduce membership on incorrectly classified samples than on correctly classified ones. This results from the fact, that model predictions are often better on training than on test data points, whereby your attack model might learn to predict incorrectly classified samples as non-members. If your model overfits the training data, this assumption might hold true often enough to make the attack seem more successful on this slice. If you wish to reduce that, pay attention to reducing your model’s overfitting.

##### Slicing: Specific classes more vulnerable: 
It seems that the membership inference attack is more successful on your class X than on the other classes. Research has shown that the class distribution (and also the distribution of data points within one class) are factors that influence the vulnerability of a class for membership inference attacks [5].

Also, small classes (belonging to minority groups) can be more prone to membership inference attacks [6]. One reason for this could be, that there is less data for that class, and therefore, the model overfits within this class. It might make sense to look into the vulnerable classes of your model again, and maybe add more data to them, use private synthetic data, or introduce privacy methods like Differential Privacy [6]. Attention, the use of Differential Privacy could have a negative influence on the performance of your model for the minority classes.


(For References, please see last box)

### TensorFlow

Now we do the same with the TensorFlow model.

In [ ]:
# Plot relative values 
labels, count = output.histogram_top_k_relative(range(10), 50)

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `tf.keras.losses.CategoricalCrossentropy`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize TensorFlow target model
target_model = Classifier(
    tf_dcti.load_dcti(), # TensorFlow DCTI
    loss=tf.keras.losses.CategoricalCrossentropy(), # Loss function of the TensorFlow target model
    nb_classes=TFCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TFCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

#### Load CIFAR10 Dataset

Again, before we can start to conduct the membership inference attacks, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the TensorFlow model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TFCIFAR10.numpy()

#### Perform Membership Inference Label Only Decision Boundary Attack

Last but not least, we want to perform a Label Only Decision Boundary Attack. We again need to train the attack model first. This attack is really time expensive, thus we only fit the attack on a very small set which is not realistic and should not be repeated in production.

In [ ]:
attack = MembershipInferenceLabelOnlyDecisionBoundaryAttack(target_model)
attack.fit(x_train[:100], y_train[:100], x_test[:20], y_test[:20], max_iter=5, max_eval=5, init_eval=5)
attack.attack(x_train[:20], y_train[:20])

#### Get human-readable attack statistics

For this attack we want a human-readable output, thus we convert it to a sting.

In [ ]:
output = attack.attack_output(
    x_train[:20], 
    y_train[:20], 
    x_train, 
    y_train, 
    x_test, 
    y_test,
    np.ones((20,))
)

str(output)

#### Explanation of the outcome:

Attack Model Accuracy:
The attack model accuracy specifies how well the membership attack model performs in predicting if a given data point was used for training the target model. Since we have a two-class classification problem that the attack model solves (member or non-member), the lowest possible accuracy is 50% (random guessing for each sample). The best accuracy is at 100% if the model predicts every data point is sees right as member or non-member.

Train-Test-Gap (difference):
If your model has a train-test-gap larger than 5%, this could be a sign that your model overfits. Overfitting can be beneficial for successful membership inference attacks [1]. Therefore, you might want to reduce it by introducing regularization methods in your training, or using specific privacy methods [2,3], such as Differential Privacy [4].

#### Perpare attack analysis

Next, we prepare our attack analysis. To initialize our attack analysis we define the Membership Inference Attack method we want to perform (in this case we use the `MembershipInferenceBlackBoxRuleBasedAttack`) and the Attack Input Data. The Attack Input Data consists of two different sets. The first contains the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The second contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model.

In [ ]:
analysis = MembershipInferenceAttackAnalysis(
    MembershipInferenceLabelOnlyDecisionBoundaryAttack, 
    AttackInputData(
        x_train[:100], 
        y_train[:100], 
        x_test[:100], 
        y_test[:100]
    )
)

#### Define the slicing

Now we can define the slicing for our analysis. The slicing defines how the data will be sliced. Each slice will then be analysed by the attack separately, so see if the inference works better on certain parts of data than on others. 

In [ ]:
slicing = Slicing(
    entire_dataset=True, 
    by_class=True, 
    by_classification_correctness=True
)

#### Perform Membership Inference Attack Analysis

Finally, we can perform our Membership Inference Attack Analysis. Therefore, we input the target model, the data that should be analysed, the membership labels (i.e. the labels which correctly describe if a data point is a member of the training dataset or not) and the splicing specification into the `analyse()` method. As a result, we get for each slice the indices of the corresponding data points, a human-readable description of the slice and the advantage score of the Membership Inference Attack (for more details about the advantage score, please read the following paper: https://arxiv.org/abs/1709.01604).

In [ ]:
result = analysis.analyse(
    target_model, 
    np.concatenate((x_train[:100], x_test[:100])), 
    np.concatenate((y_train[:100], y_test[:100])), 
    np.concatenate((np.ones(len(x_train[:100])), np.zeros(len(x_test[:100])))), 
    slicing
)

print("\n".join((str(r) for r in result)))

#### Explanation of the outcome:
##### Attacker Advantage:
The attacker advantageis a score that relies on comparing the model output on member and non-member data points. The model outputs are probability values over all classes, and they are often different on member and non-member data points. Usually, the model is more confident on member data points, because it has seen them during training. When trying to find a threshold value to tell apart member and non-member samples by their different model outputs, the attacker has interest in finding the best ratio between false positives “fpr” (non-members that are classified as members) and true positives “tpr” (members that are correctly identifies as members). 

This best ratio is calculated as the max(tpr-fpr) over all threshold values and represents the attacker advantage. 

##### Slicing: Incorrectly classified:
It is normal that the attacker is more successful to deduce membership on incorrectly classified samples than on correctly classified ones. This results from the fact, that model predictions are often better on training than on test data points, whereby your attack model might learn to predict incorrectly classified samples as non-members. If your model overfits the training data, this assumption might hold true often enough to make the attack seem more successful on this slice. If you wish to reduce that, pay attention to reducing your model’s overfitting.

##### Slicing: Specific classes more vulnerable: 
It seems that the membership inference attack is more successful on your class X than on the other classes. Research has shown that the class distribution (and also the distribution of data points within one class) are factors that influence the vulnerability of a class for membership inference attacks [5].

Also, small classes (belonging to minority groups) can be more prone to membership inference attacks [6]. One reason for this could be, that there is less data for that class, and therefore, the model overfits within this class. It might make sense to look into the vulnerable classes of your model again, and maybe add more data to them, use private synthetic data, or introduce privacy methods like Differential Privacy [6]. Attention, the use of Differential Privacy could have a negative influence on the performance of your model for the minority classes.


(For References, please see last box)

#### Compute privacy risk score

As a next step, we want to compute the privacy risk scores. To do so, we input the target model and the data points which should be evaluated to the respective function. The given data points are separated into a train and tests set. The train set contains of the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The test set contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model. As a result, we get privacy risk scores for each data point, separated into train and test scores. The resulting values indicate the probability of a data point being a member or not.

[1]S. Yeom, I. Giacomelli, M. Fredrikson, and S. Jha. \Privacy Risk in Machine Learning: Analyzing the Connection to Overfitting". In: 2018 IEEE 31st Computer Security Foundations Symposium (CSF). July 2018, pp. 268{282. doi:10.1109/CSF.2018.00027.

[2] Reza Shokri, Marco Stronati, Congzheng Song, and Vitaly Shmatikov. 2017. Mem-bership Inference Attacks Against Machine Learning Models. In2017 IEEE Sym-posium on Security and Privacy (SP). 3–18.

[3] Milad Nasr, Reza Shokri, and Amir Houmansadr. 2018. Machine Learning withMembership Privacy Using Adversarial Regularization. InProceedings of the 2018ACM SIGSAC Conference on Computer and Communications Security(Toronto,Canada)(CCS ’18). Association for Computing Machinery, New York, NY, USA,634–64

[4] Cynthia Dwork. 2006.  Differential Privacy. InAutomata, Languages and Pro-gramming, Michele Bugliesi, Bart Preneel, Vladimiro Sassone, and Ingo Wegener(Eds.). Springer Berlin Heidelberg, Berlin, Heidelberg

[5] Stacey Truex, Ling Liu, Mehmet Emre Gursoy, Lei Yu, and Wenqi Wei. 2019.Demystifying Membership Inference Attacks in Machine Learning as a Service.IEEE Transactions on Services Computing(2019)

[6] Suriyakumar, Vinith M., Nicolas Papernot, Anna Goldenberg, and Marzyeh Ghassemi. "Chasing Your Long Tails: Differentially Private Prediction in Health Care Settings." In Proceedings of the 2021 ACM Conference on Fairness, Accountability, and Transparency, pp. 723-734. 2021.

[7] Yunhui Long, Vincent Bindschaedler, Lei Wang, Diyue Bu, Xiaofeng Wang, HaixuTang, Carl A. Gunter, and Kai Chen. 2018.   Understanding Membership In-ferences on Well-Generalized Learning Models.CoRRabs/1802.04889 (2018).arXiv:1802.04889  http://arxiv.org/abs/1802.0
